In [1]:
# %pip install -q -U bitsandbytes
# %pip install datasets -U
# %pip install -q -U git+https://github.com/huggingface/transformers.git
# %pip install -q -U git+https://github.com/huggingface/peft.git
# %pip install -q -U git+https://github.com/huggingface/accelerate.git
# %pip install pandas
# %pip install wandb

In [2]:
# %pip install --upgrade transformers peft awq

# 토크나이저 및 데이터 준비

## Text Data 준비

In [3]:
from datasets import load_dataset
import pandas as pd
from huggingface_hub import login

In [4]:
import os
from dotenv import load_dotenv

load_dotenv("../keys.env")
hf_token = os.getenv("HF_TOKEN")


my_hf_key = hf_token
login(my_hf_key)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [5]:
# 모델 레포지토리
model_path = "beomi/Llama-3-Open-Ko-8B" ## "meta-llama/Meta-Llama-3.1-8B-Instruct"

# 데이터 path
data_path = "kyujinpy/KOR-OpenOrca-Platypus-v2" ## "MarkrAI/KOpen-HQ-Hermes-2.5-60K", ""DopeorNope/Ko-Optimize_Dataset""

In [6]:
# dataset 다운

data = load_dataset(data_path)

README.md:   0%|          | 0.00/13.7k [00:00<?, ?B/s]

(…)-00000-of-00001-087cb44cf03432c1.parquet:   0%|          | 0.00/38.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/44394 [00:00<?, ? examples/s]

In [7]:
df = pd.DataFrame(data['train'])

In [8]:
# 데이터셋 구성확인
df.head()

,id,input,output,instruction
0,ko_platypus.0,,"모든 가능한 결과의 확률의 합이 1$이므로, 스피너가 $C$에 착륙할 확률을 구하려...","보드 게임 스피너는 $A$, $B$, $C$로 표시된 세 부분으로 나뉩니다. 스피너..."
1,ko_platypus.1,,14명 중 6명을 선택해야 하는데 순서는 중요하지 않습니다. 이것은 순열 문제가 아...,저희 학교 수학 클럽에는 남학생 6명과 여학생 8명이 있습니다. 주 수학 경시대회...
2,ko_platypus.2,,먼저 단어에 제한을 두지 않고 4글자로 된 모든 단어의 개수를 세어봅니다. 그런 다...,"자음이 하나 이상인 4글자 단어는 $A$, $B$, $C$, $D$, $E$로 몇 ..."
3,ko_platypus.3,,주사위 중 하나 이상이 1에 나올 때만 가능합니다. 두 주사위가 모두 1이 아닐 확...,멜린다는 표준 6면 주사위 두 개를 굴려서 굴린 두 개의 숫자로 두 자리 숫자를 만...
4,ko_platypus.4,,문제를 H와 T의 시퀀스라고 생각하세요. 두 개의 T가 연속으로 나타날 수 없으므로...,p$를 공정한 동전을 반복적으로 던지는 과정에서 5$의 앞면이 나오기 전에 2$의 ...


In [9]:
sample = df.iloc[0]
print(sample.keys())
print(f"Input : \n{sample['input']}\n")
print(f"Instruction : \n{sample['instruction']}\n")
print(f"Output : \n{sample['output']}")

Index(['id', 'input', 'output', 'instruction'], dtype='object')
Input : 


Instruction : 
보드 게임 스피너는 $A$, $B$, $C$로 표시된 세 부분으로 나뉩니다. 스피너가 $A$에 떨어질 확률은 $\frac{1}{3}$이고, 스피너가 $B$에 떨어질 확률은 $\frac{5}{12}$입니다.  스피너가 $C$에 착륙할 확률은 얼마입니까? 답을 공통 분수로 표현하세요.

Output : 
모든 가능한 결과의 확률의 합이 1$이므로, 스피너가 $C$에 착륙할 확률을 구하려면 스피너가 $A$와 $B$에 착륙할 확률을 1$에서 빼야 합니다. 이를 방정식으로 쓸 수 있습니다: $P(C) = 1 - P(A) - P(B)$. P(A) = \frac{1}{3}$, $P(B) = \frac{5}{12}$라는 것을 알고 있으므로 이 값을 방정식에 대입하여 단순화할 수 있습니다. 결과는 다음과 같습니다: P(C) = 1 - \frac{1}{3} - frac{5}{12} = \frac{12}{12} - frac{4}{12} - frac{5}{12} = \frac{3}{12}$. 분자와 분모를 $3$로 나누면 이 분수를 줄일 수 있습니다: P(C) = \frac{1}{4}$입니다.


In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, Trainer, TrainingArguments, DataCollatorForSeq2Seq
import bitsandbytes as bnb
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training
)

/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
tokenizer.special_tokens_map ## pad 토큰이 없음.

{'bos_token': '<|begin_of_text|>', 'eos_token': '<|end_of_text|>'}

In [13]:
# 토크나이저 세팅: QLoRA시 pad 토큰을 eos로 설정해주기
"""
llama3에서는 pad 토큰이 사용되지 않았는데 이러한 경우 eos 토큰을 pad 토큰으로 사용함.
단, transformers 라이브러리의 trainer를 활용할 때는 문제가 없으나 trl의 SFTTrainer를 사용하는 경우 eos 토큰을 생성하지 않고 무한히 단어를 생성해버리는 문제가 있음.
"""
bos = tokenizer.bos_token_id
eos = tokenizer.eos_token_id

# tokenizer.add_special_tokens({"pad_token" : "<|reserved_special_token_0|>"})
# tokenizer.padding_side = "right"

tokenizer.pad_token_id = eos
tokenizer.padding_side = "right"

cut_off_len = 4098 ## 잘라서 사용할 sequence length
val_size = 0.005 ## 전체 데이터셋의 0.005%를 valid set으로 사용한다.
train_on_inputs = True ## 학습 단계에서 시작(질문의 첫번째 토큰)부터 끝(정답의 마지막 토큰)까지 모두 loss를 학습 -> context에 대한 이해도가 높아지게 된다.
add_eos_token = False ## tokenizing 단계에서 eos 토큰을 사용하지 않음.

In [14]:
## 알파카 템플릿
template = {
    "prompt_input": "아래는 문제를 설명하는 지시사항과, 구체적인 답변을 방식을 요구하는 입력이 함께 있는 문장입니다. 이 요청에 대해 적절하게 답변해주세요.\n###입력:{input}\n###지시사항:{instruction}\n###답변:",
    "prompt_no_input": "아래는 문제를 설명하는 지시사항입니다. 이 요청에 대해 적절하게 답변해주세요.\n###지시사항:{instruction}\n###답변:"
}

In [15]:

from typing import Union

def generate_prompt(
    instruction: str,
    input: Union[None, str] = None,
    label: Union[None, str] = None,
    verbose: bool = False
) -> str:
    """
    주어진 instruction, input, label을 사용하여 프롬프트를 생성하는 함수.

    Parameters:
    - instruction (str): 문제 설명 또는 지시사항.
    - template (dict): 입력이 있는 경우와 없는 경우의 템플릿을 포함한 딕셔너리.
    - input (str or None): 문제에 대한 구체적인 입력 (옵션).
    - label (str or None): 정답 또는 응답 (옵션).
    - verbose (bool): 생성된 프롬프트를 출력할지 여부.

    Returns:
    - str: 완성된 프롬프트.
    """
    if input:
        res = template["prompt_input"].format(instruction=instruction, input=input)
    else:
        res = template["prompt_no_input"].format(instruction=instruction)

    if label:
        res = f"{res}{label}"

    if verbose:
        print(res)

    return res


In [16]:
print(generate_prompt(df.iloc[0]['instruction']), df.iloc[0]['input'], df.iloc[0]['output'])

아래는 문제를 설명하는 지시사항입니다. 이 요청에 대해 적절하게 답변해주세요.
###지시사항:보드 게임 스피너는 $A$, $B$, $C$로 표시된 세 부분으로 나뉩니다. 스피너가 $A$에 떨어질 확률은 $\frac{1}{3}$이고, 스피너가 $B$에 떨어질 확률은 $\frac{5}{12}$입니다.  스피너가 $C$에 착륙할 확률은 얼마입니까? 답을 공통 분수로 표현하세요.
###답변:  모든 가능한 결과의 확률의 합이 1$이므로, 스피너가 $C$에 착륙할 확률을 구하려면 스피너가 $A$와 $B$에 착륙할 확률을 1$에서 빼야 합니다. 이를 방정식으로 쓸 수 있습니다: $P(C) = 1 - P(A) - P(B)$. P(A) = \frac{1}{3}$, $P(B) = \frac{5}{12}$라는 것을 알고 있으므로 이 값을 방정식에 대입하여 단순화할 수 있습니다. 결과는 다음과 같습니다: P(C) = 1 - \frac{1}{3} - frac{5}{12} = \frac{12}{12} - frac{4}{12} - frac{5}{12} = \frac{3}{12}$. 분자와 분모를 $3$로 나누면 이 분수를 줄일 수 있습니다: P(C) = \frac{1}{4}$입니다.


In [17]:
print(generate_prompt(df.iloc[1]['instruction']), df.iloc[1]['input'], df.iloc[1]['output'])

아래는 문제를 설명하는 지시사항입니다. 이 요청에 대해 적절하게 답변해주세요.
###지시사항:저희 학교 수학 클럽에는 남학생 6명과 여학생 8명이 있습니다.  주 수학 경시대회에 파견할 팀을 선발해야 합니다. 팀에 6명이 필요합니다.  제한 없이 팀을 몇 가지 방법으로 선발할 수 있나요?
###답변:  14명 중 6명을 선택해야 하는데 순서는 중요하지 않습니다. 이것은 순열 문제가 아니라 조합 문제입니다. 조합의 공식은 nCr = n! / (r! * (n-r)!)이며, 여기서 n은 총 선택의 개수이고 r은 선택의 개수입니다. 숫자를 연결하면 14C6 = 14! / (6! * 8!) = 3003.


In [18]:
def tokenize(prompt, add_eos_token=True):
   result = tokenizer(prompt,truncation=True,max_length=cut_off_len,padding=False,return_tensors=None,)
   ## 토큰화된 값의 마지막이 eos_token_id와 같지 않고
   ## input_ids의 길이가 cut_off_len보다 작고
   ## add_eos_token이 True일 때 eos 토큰을 추가하고 attention_mask에 1을 추가함.
   
   if (result["input_ids"][-1] != tokenizer.eos_token_id
       and len(result["input_ids"]) < cut_off_len
       and add_eos_token
      ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

   result["labels"] = result["input_ids"].copy() ## input_ids를 label로 사용.(Auto Regressive)
   return result

In [19]:
def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(
        data_point["instruction"],
        data_point["input"],
        data_point["output"]
    )
    
    tokenized_full_prompt = tokenize(full_prompt)
    
    ## train_on_inputs == False일 때 실행
    ## 즉, 정답문의 토큰에서만 손실이 계산되게 된다.
    if not train_on_inputs:
        user_prompt = generate_prompt(data_point["instruction"], data_point["input"])
        
        ## eos 토큰을 추가하지 않게 함.(토큰 하나를 줄이기 위함.)
        tokenized_user_prompt = tokenize(user_prompt, add_eos_token=add_eos_token)

        user_prompt_len = len(tokenized_user_prompt["input_ids"])

        if add_eos_token:
            user_prompt_len -= 1

        ## 질문에 해당하는 부분에는 -100을 곱하게 되어 학습을 하지 않게 하고, 정답 부분에서만 손실이 발생되도록 만든다.
        tokenized_full_prompt["labels"] = [-100] * user_prompt_len + tokenized_full_prompt["labels"][user_prompt_len:]

    return tokenized_full_prompt

In [20]:
if val_size > 0:
  train_val = data["train"].train_test_split(test_size=val_size, shuffle=True, seed=42)
  train_data = (train_val["train"].shuffle().map(generate_and_tokenize_prompt))
  val_data = (train_val["test"].shuffle().map(generate_and_tokenize_prompt))
  
else:
  train_data = data["train"].shuffle().map(generate_and_tokenize_prompt)
  val_data = None

Map:   0%|          | 0/44172 [00:00<?, ? examples/s]

Map:   0%|          | 0/222 [00:00<?, ? examples/s]

# Model 준비

In [21]:
# Quantization config 준비
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_storage=torch.bfloat16,
)

In [22]:
# Model 로드 하기
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config = quantization_config,
    torch_dtype = torch.bfloat16,
    device_map = {"" : 0}
)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [23]:
model = prepare_model_for_kbit_training(model) ## 양자화 학습.

In [24]:
config = LoraConfig(
    r = 16, ## 랭크
    lora_alpha = 32, ## AB를 W에 얼마나 더해줄 것인지. 가중치.
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'], ## 어텐션 모듈에 Q, K, V에 대해 Adapter 추가.
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CAUSAL_LM"
)

In [25]:
## adapter를 추가할 선형 모듈들이 무엇이 있는지 탐색.
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])

  return list(lora_module_names)

In [26]:
print('Trainable targer module:',find_all_linear_names(model))

Trainable targer module: ['q_proj', 'down_proj', 'v_proj', 'up_proj', 'o_proj', 'k_proj', 'gate_proj']


In [27]:
# QLoRA 준비
model = get_peft_model(model, config)

In [28]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Linear

In [29]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [30]:
# 파라미터 수 체크
print_trainable_parameters(model)

trainable params: 13631488 || all params: 4554231808 || trainable%: 0.29931475986915773


In [31]:
#Hyper parameter setting

output_dir='./llama_singleGPU-v1'

num_epochs = 1
micro_batch_size = 1
gradient_accumulation_steps = 16
warmup_steps = 10
learning_rate = 5e-7
group_by_length = False
optimizer = 'paged_adamw_8bit'

# adam 활용시
beta1 = 0.9
beta2 = 0.95

lr_scheduler = 'cosine'
logging_steps = 1

use_wandb = True
wandb_run_name = 'Single_GPU_Optim'

use_fp16 = False
use_bf_16 = True
evaluation_strategy = 'steps'
eval_steps = 50
save_steps = 50
save_strategy = 'steps'

In [32]:
model.gradient_checkpointing_enable()

In [33]:
trainer = Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=TrainingArguments(
    per_device_train_batch_size = micro_batch_size,
    per_device_eval_batch_size = micro_batch_size,
    gradient_accumulation_steps = gradient_accumulation_steps,
    warmup_steps = warmup_steps,
    num_train_epochs = num_epochs,
    learning_rate = learning_rate,
    adam_beta1 = beta1, # adam 활용할때 사용
    adam_beta2 = beta2, # adam 활용할때 사용
    fp16 = use_fp16,
    bf16 = use_bf_16,
    logging_steps = logging_steps,
    optim = optimizer,
    evaluation_strategy = evaluation_strategy if val_size > 0 else "no",
    save_strategy="steps",  #스텝기준으로 save
    eval_steps = eval_steps if val_size > 0 else None,
    save_steps = save_steps,
    lr_scheduler_type=lr_scheduler,
    output_dir = output_dir,
    #save_total_limit = 4,
    load_best_model_at_end = True if val_size > 0 else False ,
    group_by_length=group_by_length,
    report_to="wandb" if use_wandb else None,
    run_name=wandb_run_name if use_wandb else None,
    ),
    data_collator=DataCollatorForSeq2Seq(
        tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
        ),
    )

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
model.config.use_cache = False


trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Currently logged in as: zfbtldnz (pervin0527). Use `wandb login --relogin` to force relogin
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling 

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autoca

Step,Training Loss,Validation Loss


In [ ]:
trainer.save_model()
tokenizer.save_pretrained(output_dir)

In [ ]:
## Trainer가 저장하는 것은 전체 모델 + 어뎁터가 아니라 어뎁터만 저장하는 것이기 때문에 별도로 base_model과 학습된 어뎁터를 병합하는 과정이 필요하다.

# from peft import PeftModel

# base_model = AutoModelForCausalLM.from_pretrained(model_path, token=my_hf_key)

# merged_model = PeftModel.from_pretrained(base_model, output_dir)
# merged_model = merged_model.merge_and_unload()

# merged_model.push_to_hub("DoporNope/Single_GPU_Llama3-8B")
# tokenizer.push_to_hub("DopeorNope/Single_GPU_Llama3-8B")